<h1 align="center" style="color:blue"> Segmenting and Clustering Neighborhoods in Toronto</h1>

<h3 align="center" style="color:green"> *** Applied Data Science Capstone (Week3, course) ***</h3> 

In [67]:
# import libraries
import pandas as pd
import numpy as np

<h1 style="color:green"> Part I: Data Preparation  </h1>

## Scrape Wikipedia page

In [68]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# read HTML -> convert to DataFrame
html = pd.read_html(wiki_url)
dt = pd.DataFrame(html[0])
dt.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Transform DataFrame

### 3 columns: PostalCode, Borough, Neighborhood

In [69]:
# rename PostCode -> Postalcode
dt.rename(columns = { 'Postcode' : 'PostalCode', 'Neighbourhood' : 'Neighborhood'}, inplace=True)
dt.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is "Not assigned"

In [70]:
dt = dt[dt.Borough != 'Not assigned']
dt.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Handle "Not assigned" neighborhood

In [71]:
dt.Neighborhood.replace('Not assigned', dt.Borough, inplace=True)

### Combinating data

In [72]:
dt = dt.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
dt

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### Shape of Data Frame

In [73]:
dt.shape

(103, 3)

In [74]:
print('Number of rows: ', dt.shape[0])

Number of rows:  103


<h1 style="color:green"> Part II: Get the latitude and the longitude coordinates </h1>

## Download & Pre-process GeoSpatial data

In [75]:
# download geospatial dataset
!wget -q -O 'Geo_data.csv' https://cocl.us/Geospatial_data
!ls

Geo_data.csv


In [76]:
geo = pd.read_csv('Geo_data.csv')
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [77]:
geo.rename(columns={'Postal Code' : 'PostalCode'}, inplace=True)
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode    103 non-null object
Latitude      103 non-null float64
Longitude     103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


In [78]:
geo.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


## Merge Data

In [79]:
dt.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [80]:
dt = dt.merge(geo, on="PostalCode", how="left")
dt.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


<h1 style="color:green"> Part III: Explore and Cluster the neighborhoods in Toronto </h1>

## Load and Explore map of Toronto

In [81]:
# install libraries(if needed)
# !pip install geopy
# !pip install folium
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [82]:
# Find boroughs that contain the word Toronto
distinc_boroughs = list(dt['Borough'].unique())
print('Distinctive Boroughs: ', distinc_boroughs)

boroughs_contain_toronto = [b for b in distinc_boroughs if b.lower().find('toronto') != -1]
print('\nBoroughs with Toronto: ', boroughs_contain_toronto)

dt.head(10)

Distinctive Boroughs:  ['Scarborough', 'North York', 'East York', 'East Toronto', 'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto', 'Mississauga', 'Etobicoke']

Boroughs with Toronto:  ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [83]:
# Select 'toronto' rows 
toronto_dt = dt[dt['Borough'].isin(boroughs_contain_toronto)].reset_index(drop=True) # avoid the old index being added as a column
toronto_dt.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [84]:
# setting parameters for map
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [85]:
# create map of toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_dt['Latitude'], toronto_dt['Longitude'], toronto_dt['Borough'], toronto_dt['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

## Explore top-nearest venues and segment them

In [86]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'BZT3FSYA2V3VKFIKGA0RZC0341MKF1F4LGLZYDY2NLNAWISZ' # your Foursquare ID
CLIENT_SECRET = '0L2DQ023X3HEPVTPI1GFJULQ1HMXOUJTZASKERO1D2PJN13V' # your Foursquare Secrett
VERSION = '20180604'

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 50
RADIUS = 300

My credentails:
CLIENT_ID: BZT3FSYA2V3VKFIKGA0RZC0341MKF1F4LGLZYDY2NLNAWISZ
CLIENT_SECRET:0L2DQ023X3HEPVTPI1GFJULQ1HMXOUJTZASKERO1D2PJN13V


### Explore top 20-nearest venues using Foursquare API

In [87]:
from pandas.io.json import json_normalize
import requests # library to handle requests

# parse response data
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    
# get nearest venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
toronto_venues = getNearbyVenues(names=toronto_dt['Neighborhood'], latitudes=toronto_dt['Latitude'], longitudes=toronto_dt['Longitude'])

In [89]:
# Check size of result
print(toronto_venues.shape)
toronto_venues.head()

(1209, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
3,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [90]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton,Exhibition Place,Parkdale Village",25,25,25,25,25,25
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,18,18,18,18,18,18


In [91]:
# Find out how many unique catergories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 216 uniques categories.


### Analyze Each Neighborhood

In [92]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
toronto_onehot.shape

(1209, 216)

In [94]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.040000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.040000,0.000000,0.060000,0.020000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.020000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.02,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000


In [95]:
toronto_grouped.shape

(39, 216)

In [96]:
# For each neighborhood, list top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0          Steakhouse  0.06
1                Café  0.06
2  Seafood Restaurant  0.04
3      Breakfast Spot  0.04
4         Pizza Place  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1            Beer Bar  0.04
2              Bakery  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0       Nightclub  0.12
1            Café  0.12
2     Coffee Shop  0.08
3  Breakfast Spot  0.08
4    Intersection  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0            Smoke Shop  0.06
1         Garden Center  0.06
2            Skate Park  0.06
3  Fast Food Restaurant  0.06
4         Burrito Place  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airpor

In [97]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [98]:
# display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Steakhouse,American Restaurant,Seafood Restaurant,Hotel,Breakfast Spot,Bar,Pizza Place,Asian Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Seafood Restaurant,Cheese Shop,Bakery,Farmers Market,Cocktail Bar,Beer Bar,Café,Restaurant,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Nightclub,Café,Coffee Shop,Breakfast Spot,Bakery,Climbing Gym,Stadium,Bar,Italian Restaurant,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Skate Park,Recording Studio,Fast Food Restaurant,Light Rail Station,Auto Workshop,Farmers Market,Restaurant,Burrito Place,Spa,Pizza Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court


### Cluster Neighborhoods

In [99]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
# cluster the neighborhood into 5 clusters.
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_dt

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Other Great Outdoors,Trail,Health Food Store,Pub,Wings Joint,Cuban Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Bookstore,Caribbean Restaurant,Dessert Shop,Spa,Pub
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Pub,Sushi Restaurant,Brewery,Liquor Store,Burrito Place,Italian Restaurant,Intersection,Ice Cream Shop,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Gym / Fitness Center,Bus Line,Swim School,Wings Joint,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run


In [101]:
# visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Cluster

In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Wings Joint,Cupcake Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner


In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Restaurant,Playground,Wings Joint,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
